# Template 学习

In [1]:
# 字符模板
# string template
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name}，帮我起1个具有{county}特色的{sex}名字")
prompts = prompt.format(name="算命大师", county="法国", sex="女孩")
print(prompts)

你是一个算命大师，帮我起1个具有法国特色的女孩名字


In [2]:
# 对话模板具有结构，chatmodels
# chat template with structure
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个起名大师，你的名字叫{name}。"),
    ("human", "你好{name},你感觉如何？"),
    ("ai", "你好！我叫什么名字？"),
    ("human", "{user_input}")
])

chats = chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢？")
print(chats)

[SystemMessage(content='你是一个起名大师，你的名字叫陈大师。', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好陈大师,你感觉如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我叫什么名字？', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的爸爸是谁呢？', additional_kwargs={}, response_metadata={})]


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个厉害的人工智能助手"),
    MessagesPlaceholder("msgs")
    # "placeholder", "(msgs)"
])

result = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

print(result)

messages=[SystemMessage(content='你是一个厉害的人工智能助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


In [6]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 创建系统消息
system_message = SystemMessage(
    content="你是一个有用的助手",
    additional_kwargs={"大师姓名":"陈瞎子"}
)

# 创建人类消息
human_message = HumanMessage(content="hi!")

# 创建AI消息
ai_message = AIMessage(content="我有什么可以帮助你的？")

[system_message, human_message, ai_message]
# 打印消息内容
# print(system_message)
# print(human_message)
# print(ai_message)

[SystemMessage(content='你是一个有用的助手', additional_kwargs={'大师姓名': '陈瞎子'}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我有什么可以帮助你的？', additional_kwargs={}, response_metadata={})]

In [8]:
#函数大师：根据函数名称，查找函数代码，并给出中文的代码说明
# Function Master: Given a function name, find the function code and provide a Chinese code explanation
from langchain_core.prompts import StringPromptTemplate
from langchain_deepseek import ChatDeepSeek
# 导入环境变量加载工具
from dotenv import load_dotenv
import os

# 加载.env文件中的环境变量
load_dotenv()

#定义一个简单的函数作为示例效果
# Define a simple function as an example
def hello_world(abc):
    print("hello, world!")
    return abc


PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：{function_name}
源代码：
{source_code}
代码解释：
"""

import inspect


def get_source_code(function_name):
    #获得源代码
    # Get the source code
    return inspect.getsource(function_name)

#自定义的模板class
# Custom template class
class CustomPrompt(StringPromptTemplate):


    def format(self, **kwargs) -> str:
        #获得源代码
        # Get the source code 获得函数名
        source_code = get_source_code(kwargs["function_name"])

        #生成提示词模板
        # Generate the prompt template
        prompt = PROMPT.format(
            function_name=kwargs["function_name"], source_code=source_code
        )

        return prompt


#使用自定义的提示词模板，而不是类似对话提示词模板
a = CustomPrompt(input_variables=["function_name"])

pm = a.format(function_name=hello_world)
print("格式化之后的提示词为====")
print(pm)

#和LLM连接起来
# Connect to LLM
# 创建DeepSeek 聊天模型实例
llm = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

msg = llm.invoke(pm)
print(msg.content)

格式化之后的提示词为====
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：<function hello_world at 0x000002191A375C60>
源代码：
def hello_world(abc):
    print("hello, world!")
    return abc

代码解释：

- **函数名称**：`hello_world`

- **源代码**：
```python
def hello_world(abc):
    print("hello, world!")
    return abc
```

- **中文解释**：
  1. 这是一个名为 `hello_world` 的函数，接受一个参数 `abc`
  2. 函数执行时会打印固定字符串 "hello, world!" 到控制台
  3. 最后将传入的参数 `abc` 原样返回给调用者
  4. 函数本身不改变输入参数的值，只是在其前后添加了打印操作


In [10]:
# zeroshot 会导致低质量回答
from langchain_deepseek import ChatDeepSeek
# 导入环境变量加载工具
from dotenv import load_dotenv
import os

# 加载.env文件中的环境变量
load_dotenv()

# 创建DeepSeek 聊天模型实例
llm = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

res = llm.invoke("What is 2 √ 9?")
print(res.content)

Let's break this down.

The expression is:

\[
2 \sqrt{9}
\]

1. First, compute \(\sqrt{9}\):
\[
\sqrt{9} = 3
\]
because \(3^2 = 9\).

2. Then multiply:
\[
2 \times 3 = 6
\]

So the answer is:

\[
\boxed{6}
\]


In [16]:
from langchain_deepseek import ChatDeepSeek
# 导入环境变量加载工具
from dotenv import load_dotenv
import os

# 加载.env文件中的环境变量
load_dotenv()

# 创建DeepSeek 聊天模型实例
model = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 增加示例组
examples = [
    {"input": "2 √ 2", "output": "4"},
    {"input": "2 √ 3", "output": "5"},
]
# 构造提示词模板
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
# 组合示例与提示词
few_Shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# 打印提示词模板
print(few_Shot_prompt.invoke({}).to_messages())

# 最终提示词
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位神奇的数学奇才"),
        few_Shot_prompt,
        ("human", "{input}"),
    ]
)

# 重新提问
chain = final_prompt | model
result = chain.invoke({"input": "What is 2 √ 9?"})
print(result.content)
#
# res = model.invoke("What is 2 √ 9?")
# print(res)
# #增加示例
# #使用FewShotChatMessagePromptTemplate
# from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
# #增加示例组
# examples = [
#     {"input": "2 √ 2", "output": "4"},
#     {"input": "2 √ 3", "output": "5"},
# ]
# #构造提示词模板
# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("human", "{input}"),
#         ("ai", "{output}"),
#     ]
# )
# #组合示例与提示词
# few_Shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
# )
# #打印提示词模板
# print("组合之后的提示词为===）")
# print(few_Shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2 √ 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 √ 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]
6
